## Cryptographic protocol demonstration
#### Installing libraries
'pip install pycryptodome'

'pip install cryptography'


This code imports libraries and modules for cryptographic operations, including hashing, RSA encryption, serialization, and time management, enabling secure data handling and encryption tasks in Python.

In [105]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import utils
from cryptography.exceptions import InvalidSignature
from cryptography.hazmat.backends import default_backend

from Crypto.Cipher import PKCS1_OAEP

import time
import secrets
import hashlib
from datetime import datetime

#### Preparing the message for transmission

In [106]:
message = "The company website has not limited the number of transactions a single user or device can perform in a given period of time. The transactions/time should be above the actual business requirement, but low enough to deter automated attacks"

Creating timestamp using the current time in seconds in Unix format

In [107]:
# Creates a timestamp using the current time in seconds in Unix format
timestamp = int(time.time())
print(f"Timestamp: {timestamp}")

Timestamp: 1710353181


Generating a nonce as a 32-character hex string

In [108]:
nonce_hex = secrets.token_hex(16)  
print(f"Nonce (hex): {nonce_hex}")

#nonce_storage = []



Nonce (hex): e3389d582c82f7329b7a0597850331a0


Concatenating the string with a delimiter (:)

In [109]:
combined = f"{nonce_hex}:{timestamp}:{message}"
print("Combined string:", combined)

Combined string: e3389d582c82f7329b7a0597850331a0:1710353181:The company website has not limited the number of transactions a single user or device can perform in a given period of time. The transactions/time should be above the actual business requirement, but low enough to deter automated attacks


Encoding the combined string to bytes, then hashing it using SHA-256

In [110]:
hashed = hashlib.sha256(combined.encode()).hexdigest()

print("SHA-256 Hash:", hashed)

SHA-256 Hash: cb087ff7dc0915e93ccbb1b80c8ebd3ff72a6bebe9d5035ebf4c85448ce5cb26


This function generates an RSA private and public key pair.
The public_exponent is set to 65537, which is a common choice for RSA encryption as it's a prime number that ensures efficiency and security in the encryption process.
The key size of 2048 bits is selected to provide a good balance between security and performance, ensuring it is sufficiently secure for most applications.
    
The function returns a tuple containing the RSA private key and public key.


In [111]:
def generate_rsa_keys():
    
    private_key = rsa.generate_private_key(
        public_exponent=65537,  
        key_size=2048,         
        backend=default_backend()
    )

    public_key = private_key.public_key()
    
    return private_key, public_key

   
This function serializes the RSA private and public keys to PEM format for storage or transmission. PEM format is used. This is a Base64 encoded message with header and footer to mark the beginning
and end of the key material which makes it suitable for text-based transmission.
    
Arguments:

private_key: The RSA private key to be serialized.

public_key: The RSA public key to be serialized.
    
The function then returns a tuple containing the serialized private and public keys in PEM format.
 

In [112]:
def serialize_rsa_keys(private_key, public_key):

    pem_private_key = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()  # Indicates no encryption
    )
    
    pem_public_key = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    
    return pem_private_key, pem_public_key

This function is used sign hashed data using the RSA private key.
    
It uses PSS (Probabilistic Signature Scheme) padding, which is recommended
for new applications due to its security properties. The SHA-256 hash function is used
for hashing the data, providing a strong level of integrity.
    
Arguments:

private_key: The RSA private key used for signing.

hashed: The hashed data which is to be signed
    
The function then returns the digital signature of the hashed data.


In [113]:
def sign_data_with_private_key(private_key, hashed):
   

    signature = private_key.sign(
        hashed,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),  # Mask Generation Function
            salt_length=padding.PSS.MAX_LENGTH  # Maximum salt length
        ),
        hashes.SHA256()  # Hash function for signing
    )
    
    return signature

This code then uses the functions to generate a pair of RSA keys for Alice, it serializes these keys into the PEM format for storage or transmission, and then signs a hashed message using Alice's private key to create a digital signature.

In [114]:
alice_private_key, alice_public_key = generate_rsa_keys()

pem_private_key, pem_public_key = serialize_rsa_keys(alice_private_key, alice_public_key)

signature = sign_data_with_private_key(alice_private_key, hashed.encode())

The data is then prepared for transmission by combining the original message string (M:T:N) and the digital signiture into a signal string. The string is printed to simulate Bob transmitting it.

In [115]:

transmission_data = {
    'message': combined,
    'signature': signature,
}

print(transmission_data)

{'message': 'e3389d582c82f7329b7a0597850331a0:1710353181:The company website has not limited the number of transactions a single user or device can perform in a given period of time. The transactions/time should be above the actual business requirement, but low enough to deter automated attacks', 'signature': b'\x0c\xea\'\xee\xb6\xee\x03Z\x13X\xafD~y\xee\xcf|T\x06\xda\xc3qOMs\xfa\xea\xd3\xf0\x01\x12\xf1<\'\x81\xc55\xab9;\xd4\t\x99\xfe_#WR\xe1\x87NE\x8f\xb8\x1c\x15\xfd\x06ev\xb3\xe8\xcdx\xb0<\xde\xeae\xeb_\x1d\xc4\xb5UQl2\x1bE\xc7\x97\xc6\xda\xe6=\x1b\x1a\xf0\xab\x00T\x17L^\xf1\x9f*)[\xbd\xf7\xdb\x12\x15\xfe\x93M\xe5-\xfe\x8d\xda\xcf\xb8_\x91\xd0\xb5\xcc\xb1\x83\x99\xa6\xe6U\xcd~\\\x81\xecG\x81\xbf~\x16\x91\xa7sy\xeb\xe5^\x1d\xbf\x8f`vR\x8e<u\x98\xc8"N\xa7\xf1\xb5\xb12.\x93\x1d\xe6\x7f\x8ay\xecg\x05\x9d\x1e\x10\xe5\xc1\xa6\xaa\x15\xea~(\xef\r\x17#f\xbd.\xf5\xaa\xed\xf5-g!\xa8\x1cP\xce\xdd\xa3\xe0\x90\xb4#%a_\xab\xb9\xc6\xc6\xbc\xad-@~Q\x8f\xab\x12\xd03P\xbc\xf8\\\xdd\t\x9c\xe4\xb2\x08\x

#### Recieving the message

This function is used to verify the digital signiture of the recieved message using the prevously exchanged public key from Alice. This is done by decrypting the digital signiture using the public key to return a hash value. Bob then hashes the recieved message string and compares the two hashes.
If the hashes are identical then the function returns true such that the message hasn't been tampered with.
The function return false with an InvalidSigniture if these values are not matching.

In [116]:
def verify_signature(received_message, signature, public_key):
    try:
        public_key.verify(
            signature,
            received_message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except InvalidSignature:
        return False

This function checks the timestamp against the current time and determines if it is within the window of 300 seconds.

In [117]:
def is_within_time_window(received_timestamp, window=300):
    current_time = int(time.time())
    return current_time - received_timestamp <= window

This function is used to check if the nonce if unique, in order to counter against the reusing of nonces. Bob's previously recieved nonces are stored in a list, 'nonce_storage' and the the recieved nonce is comapred to this list to see if it has been used prior. If the nonce is not present in the list then the verification returns true otherwise it returns false.

In [118]:
def is_nonce_unique(nonce, nonce_storage):
    if nonce not in nonce_storage:
        nonce_storage.append(nonce)
        return True
    return False


This code simulates Bob verifying a message received from Alice. Using the previously described functions, it checks the message's signature for authenticity, confirms the timestamp is current, and ensures the nonce is unique to prevent replay attacks. Results of these verifications are printed out.

In [119]:
# Simulated received data that Bob recieves from Alice
received_data = transmission_data 

# Verify the signature of the received message
received_hashed_message = hashlib.sha256(received_data['message'].encode()).hexdigest()
verification_result = verify_signature(received_hashed_message.encode(), received_data['signature'], alice_public_key)

# Extract the timestamp from the received message
received_message = received_data['message']
nonce, received_timestamp_str, _ = received_message.split(':', 2)
received_timestamp = int(received_timestamp_str)

# Check if the received timestamp is within the allowed window
timestamp_verification_result = is_within_time_window(received_timestamp)

#Extract the nonce from the received message
nonce = received_data['message'].split(':', 1)[0] 
# Check if the nonce is unique
nonce_verification_result = is_nonce_unique(nonce, nonce_storage)

print(f"Signature verification result: {verification_result}")
print(f"Timestamp verification result: {timestamp_verification_result}")
print(f"Nonce verification result: {nonce_verification_result}")






Signature verification result: True
Timestamp verification result: True
Nonce verification result: True


This code then extracts and processes parts of a received message: it retrieves the nonce, timestamp, and the actual message content. It converts the timestamp from a string to an integer, then formats it into a human-readable date. Depending on previously obtained verification results, it either confirms the message's authenticity, uniqueness, and timeliness, or it indicates the specific reason for verification failure related to the signature, timestamp, or nonce.

In [120]:
received_message = received_data['message']

parts = received_message.split(':')
nonce = parts[0]
timestamp_str = parts[1]
actual_message = ':'.join(parts[2:])


timestamp = int(timestamp_str)


dt_object = datetime.fromtimestamp(timestamp)
readable_date = dt_object.strftime('%Y-%m-%d %H:%M:%S')

if verification_result:
    if timestamp_verification_result:
        if nonce_verification_result:
            print("The message is authentic, unique, and within the time window.")
            print(f"Nonce: {nonce}")
            print(f"Readable Timestamp: {readable_date}")
            print(f"Message: {actual_message}")
        else:
            print("The message failed verification: The nonce has already been used.")
    else:
        print("The message failed verification: The timestamp is outside the acceptable window.")
else:
    print("The message failed verification: The digital signature does not match.")


The message is authentic, unique, and within the time window.
Nonce: e3389d582c82f7329b7a0597850331a0
Readable Timestamp: 2024-03-13 18:06:21
Message: The company website has not limited the number of transactions a single user or device can perform in a given period of time. The transactions/time should be above the actual business requirement, but low enough to deter automated attacks


In [121]:
print(f"{nonce_storage}")

['ed3d3e3583885411124ea68693d890a1', 'fe29c2912135cdc8ac8fd344a43e10f0', 'e7d4cce4793a96299b788f2c67bc5dd7', 'e3389d582c82f7329b7a0597850331a0']
